In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain.chat_models import ChatOpenAI

from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

In [7]:
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
    """
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk

    Human: {example_question}
    You: {example_answer}
    """
)

start  = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You: """
)

final = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
    """
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts
)

full_prompt.format(
    character='Pirate',
    example_question='What is your location?',
    example_answer='Arrrrg! That is a secret!! Arg arg!!',
    question="What is your fav food?"
)

'\n    \n    You are a role playing assistant.\n    And you are impersonating a Pirate\n    \n\n    \n    This is an example of how you talk\n\n    Human: What is your location?\n    You: Arrrrg! That is a secret!! Arg arg!!\n    \n\n    \n    Start now!\n\n    Human: What is your fav food?\n    You: \n    '